In [54]:
import urllib3
from bs4 import BeautifulSoup
import bs4
import requests
import re
import time
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

BUFFER_TIME = 5
def extract_source(url):
    agent = {"User-Agent":"Mozilla/5.0"}
    source=requests.get(url, headers=agent).text
    return source

In [55]:
def getData(source):
    soup=bs4.BeautifulSoup(source, 'lxml')
    stocks = soup.findAll('tr')
    ids = []
    names = []
    prices = []
    changes = []
    deltas = []
    for stock in stocks:
        td = stock.findAll('td')
        i = 0
        vals = []
        for d in td:
            txt = d.get_text()
            if txt == " " or txt == "" or txt == '' or txt == 'N/A':
                continue
            else:
                vals.append(txt)
        if vals == []:
            del vals
            continue
        else:
            ids.append(vals[0])
            names.append(vals[1])
            prices.append(vals[2])
            changes.append(vals[3])
            deltas.append(vals[4])
    dct_df = {"Identifier":ids,
              "Name":names,
              "Price":prices,
              "Change":changes,
              "Percent Change":deltas}
    df = pd.DataFrame(dct_df, columns = ["Identifier",
                                         "Name",
                                         "Price",
                                         "Change",
                                         "Percent Change"])
    return df

In [64]:
dfs = []
while 1:
    # retrive current moment's data
    offset = 0
    count = 0
    og_df = getData(extract_source('https://finance.yahoo.com/most-active?count=100&offset=' + str(offset)))
    while count <1:
        offset+=100
        count+=1
        #create dataframe and add it to log of dataframes
        og_df = pd.concat([og_df,
                           getData(extract_source\
                                   ('https://finance.yahoo.com/most-active?count=100&offset='+ str(offset)))])
    dfs.append(og_df)
    print(og_df)
    time.sleep(BUFFER_TIME)

   Identifier                                               Name  Price  \
0         AAL                       American Airlines Group Inc.  17.41   
1        NCLH                Norwegian Cruise Line Holdings Ltd.  21.37   
2           F                                 Ford Motor Company   6.93   
3         NIO                                           NIO Inc.   6.39   
4         UAL                     United Airlines Holdings, Inc.  40.70   
..        ...                                                ...    ...   
16        TSM  Taiwan Semiconductor Manufacturing Company Lim...  57.63   
17        NEM                                Newmont Corporation  56.22   
18        WPX                                   WPX Energy, Inc.   7.32   
19        ING                                     ING Groep N.V.   7.76   
20       LYFT                                         Lyft, Inc.  38.95   

   Change Percent Change  
0   -1.14         -6.17%  
1   -2.76        -11.44%  
2   -0.30         

KeyboardInterrupt: 